In [83]:
#sqldf() provides a convenient interface of running SQL statement on data frames. 
#Similarly, Python also offers multiple ways to interact between SQL and Pandas DataFrames by 
#leveraging the lightweight SQLite engine.

#import the library SQLite
import sqlite3


In [17]:
# generate some sample numbers
def fib(n):
    a, b = 0, 1

    for _ in range(n):
        yield a
        a, b = b, a + b


In [18]:
#establish a conenction
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()


In [19]:
# check if the DB name exists
with connection:
    cursor.execute('''CREATE TABLE IF NOT EXISTS fib (
                            calculated_value INTEGER)''')
    cursor.executemany('INSERT INTO fib VALUES (?)',
                       [(str(x),) for x in fib(10)])


In [20]:
#run a sample query
cursor.execute('SELECT * FROM fib')
print(cursor.fetchall())

[(0,), (1,), (1,), (2,), (3,), (5,), (8,), (13,), (21,), (34,)]


In [14]:
connection.close()

In [23]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
#design a table in the sqlite db
sql = """
    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );"""

_ = cur.execute(sql)

In [25]:
print(sql)


    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );


In [26]:
sql = "insert into sensor(sid, name, notes) values (%d, '%s', '%s');"

In [27]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [28]:
for (uid, name, notes) in [(201, 'Alpha', 'Sensor for weight'), \
                           (202, 'Beta', 'Sensor for conductivity'),
                           (203, 'Gamma', 'Sensor for surface oxides'),
                           (204, 'Delta', 'Sensor for length'),
                           (205, 'Epsilon', 'Sensor for x-ray'),
                           (206, 'Zeta', 'Color checker 9000'),
                           (207, 'Eta', 'Ultra-violet detector'), ]:
    cur.execute(sql % (uid, name, notes))


In [29]:
print(sql)

insert into sensor(sid, name, notes) values (%d, '%s', '%s');


In [53]:
import numpy as np
from collections import defaultdict
import gc
import time
from pandas import DataFrame
from pandas.util.testing import rands
import random


In [54]:
N = 10000

In [56]:
indices = np.array([rands(10) for _ in range(N)], dtype='O')
indices2 = np.array([rands(10) for _ in range(N)], dtype='O')
key = np.tile(indices[:8000], 10)
key2 = np.tile(indices2[:8000], 10)

In [57]:
left = DataFrame({'key' : key, 'key2':key2,
                  'value' : np.random.randn(80000)})

In [59]:
left.head()

,key,key2,value
0,YBOnfoSeQl,S1yHvLhEvt,-0.024912
1,Iq2HVee4P1,bP5DAe3xA8,0.926543
2,Jawh8RlJ6I,a2W0lGTv6t,-0.616354
3,fMibW63YLw,qUK5LlIXom,-0.222069
4,cFZUwReCpb,YVtc87ixLx,-1.034871


In [60]:
right = DataFrame({'key': indices[2000:], 'key2':indices2[2000:],
                   'value2' : np.random.randn(8000)})


In [61]:
right.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [62]:
right2 = right.append(right, ignore_index=True)
right = right2


In [63]:
right.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [64]:
right2.head()

,key,key2,value2
0,8GDq1LbK8Q,Riobsv5WIz,-0.983521
1,KRh4t1HBVk,4nP5qnpLdw,-0.710309
2,NB8bhTas7x,IRHnka1FVh,1.050322
3,xL2IYzW6bB,XMeyWpwrab,0.792583
4,xLD0Ihjru5,Q5OY980ovi,0.707779


In [65]:
random.shuffle(key2)

In [66]:
indices2 = indices.copy()

In [67]:
random.shuffle(indices2)

In [68]:
# Prepare Database
import sqlite3
create_sql_indexes = False

In [69]:
conn = sqlite3.connect(':memory:')

In [70]:
conn.execute('create table left( key varchar(10), key2 varchar(10), value int);')

In [71]:
conn.execute('create table right( key varchar(10), key2 varchar(10), value2 int);')

In [72]:
conn.executemany('insert into left values (?, ?, ?)',
                 zip(key, key2, left['value']))

In [73]:
conn.executemany('insert into right values (?, ?, ?)',
                 zip(right['key'], right['key2'], right['value2']))


In [74]:
# Create Indices
if create_sql_indexes:
    conn.execute('create index left_ix on left(key, key2)')
    conn.execute('create index right_ix on right(key, key2)')


In [75]:
join_methods = ['inner', 'left outer', 'left'] # others not supported

In [76]:
join_methods

['inner', 'left outer', 'left']

In [77]:
sql_results = DataFrame(index=join_methods, columns=[False])

In [78]:
sql_results

,False
inner,NaN
left outer,NaN
left,NaN


In [79]:
niter = 5

In [81]:
for sort in [False]:
    for join_method in join_methods:
        sql = """select *
        from left
           %s join right
             on left.key=right.key
               and left.key2 = right.key2""" % join_method
        if sort:
            sql = '%s order by key, key2' % sql
        f = lambda: list(conn.execute(sql)) # list fetches results
        g = lambda: conn.execute(sql) # list fetches results
        gc.disable()
        start = time.time()
        for _ in range(niter):
            f()
        elapsed = (time.time() - start) / niter
        gc.enable()
        sql_results[sort][join_method] = elapsed
sql_results.columns = ['sqlite3'] # ['dont_sort', 'sort']
sql_results.index = ['inner', 'outer', 'left']

In [82]:
sql_results

,sqlite3
inner,0.165884
outer,0.409653
left,0.425291


In [84]:
#advantages of pysqldf over pandasql are two-fold. 

#1. pysqldf is 2 – 3 times faster than pandasql. 
#2. pysqldf supports new function definitions, which is not available in pandasql. 

#However, the generic python interface to an in-memory SQLite database can be more efficient and 
#flexible than both pysqldf and pandasql

In [109]:
import sqlalchemy

In [110]:
sqlalchemy.__version__

'1.1.13'

#The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables. It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.

In [111]:
from sqlalchemy import create_engine

In [112]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [113]:
#The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module. 

In [114]:
#The return value of create_engine() is an instance of Engine, and it represents the core interface to the database, 
#adapted through a dialect that handles the details of the database and DBAPI in use. In this case the SQLite dialect will 
#interpret instructions to the Python built-in sqlite3 module.



In [115]:
from sqlalchemy.ext.declarative import declarative_base

In [116]:
Base = declarative_base()

In [117]:
from sqlalchemy import Column, Integer, String

In [118]:
class User(Base):
...     __tablename__ = 'users'
...
...     id = Column(Integer, primary_key=True)
...     name = Column(String)
...     fullname = Column(String)
...     password = Column(String)
...
...     def __repr__(self):
...        return "<User(name='%s', fullname='%s', password='%s')>" % (
...                             self.name, self.fullname, self.password)

In [120]:
#With our User class constructed via the Declarative system, we have defined information about our table,
#known as table metadata. The object used by SQLAlchemy to represent this information for a specific table 
#is called the Table object, 
#and here Declarative has made one for us.

In [122]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

#The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database. As our SQLite database does not actually have a users table present, we can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. 

In [123]:
Base.metadata.create_all(engine)

2018-03-18 07:28:37,666 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2018-03-18 07:28:37,768 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-03-18 07:28:37,774 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)




2018-03-18 07:28:37,799 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-03-18 07:28:37,803 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [124]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [125]:
ed_user.name

'ed'

In [126]:
ed_user.password

'edspassword'

In [127]:
str(ed_user.id)

'None'

In [128]:
from sqlalchemy.orm import sessionmaker

In [129]:
Session = sessionmaker(bind=engine)

In [130]:
#In the case where your application does not yet have an Engine when you define your module-level objects

In [131]:
Session = sessionmaker()

In [132]:
#when you create your engine with create_engine(), connect it to the Session using configure()

In [133]:
Session.configure(bind=engine) 

In [134]:
session = Session()

In [135]:
#Adding and Updating Objects

In [136]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [137]:
session.add(ed_user)

In [138]:
our_user = session.query(User).filter_by(name='ed').first() 

2018-03-18 07:32:42,998 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:32:43,005 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2018-03-18 07:32:43,007 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones', 'edspassword')


2018-03-18 07:32:43,013 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2018-03-18 07:32:43,015 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('ed', 1, 0)


In [139]:
our_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [140]:
ed_user is our_user

True

In [141]:
#We can add more User objects at once using add_all():

In [142]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', password='foobar'),
...     User(name='mary', fullname='Mary Contrary', password='xxg527'),
...     User(name='fred', fullname='Fred Flinstone', password='blah')])

In [143]:
#the password for Ed isn’t too secure, so lets change it:

In [144]:
ed_user.password = 'f8s7ccs'

In [145]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

In [146]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])

#We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). 

In [147]:
session.commit()

2018-03-18 07:35:15,263 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET password=? WHERE users.id = ?


2018-03-18 07:35:15,269 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 1)


INFO:sqlalchemy.engine.base.Engine:('f8s7ccs', 1)


2018-03-18 07:35:15,272 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2018-03-18 07:35:15,287 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')


INFO:sqlalchemy.engine.base.Engine:('fred', 'Fred Flinstone', 'blah')


2018-03-18 07:35:15,290 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [148]:
#commit() flushes the remaining changes to the database, and commits the transaction. 

In [149]:
#Since the Session works within a transaction, we can roll back changes made too.

In [150]:
ed_user.name = 'Edwardo'

In [151]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [152]:
session.add(fake_user)

In [153]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2018-03-18 07:36:35,665 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:36:35,670 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2018-03-18 07:36:35,673 INFO sqlalchemy.engine.base.Engine (1,)


...

2018-03-18 07:36:35,690 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2018-03-18 07:36:35,693 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [154]:
session.rollback()

2018-03-18 07:36:49,342 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [155]:
ed_user.name

2018-03-18 07:36:56,242 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2018-03-18 07:36:56,247 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2018-03-18 07:36:56,250 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)


'ed'

In [156]:
fake_user in session

False

In [157]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2018-03-18 07:37:20,117 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2018-03-18 07:37:20,121 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

#A Query object is created using the query() method on Session. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors.

In [158]:
for instance in session.query(User).order_by(User.id):
...     print(instance.name, instance.fullname)

2018-03-18 07:37:41,250 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


2018-03-18 07:37:41,255 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [159]:
for name, fullname in session.query(User.name, User.fullname):
...     print(name, fullname)

2018-03-18 07:38:26,315 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


2018-03-18 07:38:26,319 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [160]:
for row in session.query(User, User.name).all():
...    print(row.User, row.name)

2018-03-18 07:38:46,126 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


2018-03-18 07:38:46,130 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


In [161]:
for row in session.query(User.name.label('name_label')).all():
...    print(row.name_label)

2018-03-18 07:38:55,685 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS name_label 
FROM users


2018-03-18 07:38:55,688 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed
wendy
mary
fred


In [163]:
#The name given to a full entity such as User, assuming that multiple entities are present in the call to query(), 
#can be controlled using aliased() :


In [164]:
from sqlalchemy.orm import aliased

In [165]:
user_alias = aliased(User, name='user_alias')

In [166]:
for row in session.query(user_alias, user_alias.name).all():
...    print(row.user_alias)

2018-03-18 07:39:48,124 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


INFO:sqlalchemy.engine.base.Engine:SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


2018-03-18 07:39:48,128 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [167]:
#Basic operations with Query include issuing LIMIT and OFFSET, most conveniently using Python array slices and 
#typically in conjunction with ORDER BY:


In [168]:
for u in session.query(User).order_by(User.id)[1:3]:
...    print(u)

2018-03-18 07:40:26,264 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


2018-03-18 07:40:26,268 INFO sqlalchemy.engine.base.Engine (2, 1)


INFO:sqlalchemy.engine.base.Engine:(2, 1)


<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


In [169]:
for name, in session.query(User.name).\
...             filter_by(fullname='Ed Jones'):
...    print(name)

2018-03-18 07:40:40,993 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2018-03-18 07:40:40,998 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ed


In [170]:
for name, in session.query(User.name).\
...             filter(User.fullname=='Ed Jones'):
...    print(name)

2018-03-18 07:40:51,774 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2018-03-18 07:40:51,779 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ed


In [171]:
for user in session.query(User).\
...          filter(User.name=='ed').\
...          filter(User.fullname=='Ed Jones'):
...    print(user)

2018-03-18 07:41:12,290 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


2018-03-18 07:41:12,295 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones')


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>


In [172]:
#Here’s a rundown of some of the most common operators used in filter():


In [175]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [176]:
query.all() #all() returns a list:

2018-03-18 07:42:48,471 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id


2018-03-18 07:42:48,475 INFO sqlalchemy.engine.base.Engine ('%ed',)


INFO:sqlalchemy.engine.base.Engine:('%ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fred', fullname='Fred Flinstone', password='blah')>]

In [177]:
query.first()

2018-03-18 07:43:15,732 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?


2018-03-18 07:43:15,736 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>

In [179]:
from sqlalchemy import text

In [180]:
#Literal strings can be used flexibly with Query, by specifying their use with the text() construct

In [181]:
for user in session.query(User).\
...             filter(text("id<224")).\
...             order_by(text("id")).all():
...     print(user.name)

2018-03-18 07:44:24,145 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY id


2018-03-18 07:44:24,149 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ed
wendy
mary
fred


In [182]:
session.query(User).filter(text("id<:value and name=:name")).\
...     params(value=224, name='fred').order_by(User.id).one()

2018-03-18 07:44:41,027 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id


2018-03-18 07:44:41,034 INFO sqlalchemy.engine.base.Engine (224, 'fred')


INFO:sqlalchemy.engine.base.Engine:(224, 'fred')


<User(name='fred', fullname='Fred Flinstone', password='blah')>

In [184]:
#To use an entirely string-based statement, a text() construct representing a complete 
#statement can be passed to from_statement(). 

In [183]:
session.query(User).from_statement(
...                     text("SELECT * FROM users where name=:name")).\
...                     params(name='ed').all()

2018-03-18 07:44:58,385 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?


INFO:sqlalchemy.engine.base.Engine:SELECT * FROM users where name=?


2018-03-18 07:44:58,389 INFO sqlalchemy.engine.base.Engine ('ed',)


INFO:sqlalchemy.engine.base.Engine:('ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

In [185]:
stmt = text("SELECT name, id, fullname, password "
...             "FROM users where name=:name")

In [186]:
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)

In [187]:
session.query(User).from_statement(stmt).params(name='ed').all()

2018-03-18 07:46:00,195 INFO sqlalchemy.engine.base.Engine SELECT name, id, fullname, password FROM users where name=?


INFO:sqlalchemy.engine.base.Engine:SELECT name, id, fullname, password FROM users where name=?


2018-03-18 07:46:00,200 INFO sqlalchemy.engine.base.Engine ('ed',)


INFO:sqlalchemy.engine.base.Engine:('ed',)


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

In [188]:
#The count() method is used to determine how many rows the SQL statement would return. 

In [189]:
session.query(User).filter(User.name.like('%ed')).count()

2018-03-18 07:46:33,417 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1


2018-03-18 07:46:33,422 INFO sqlalchemy.engine.base.Engine ('%ed',)


INFO:sqlalchemy.engine.base.Engine:('%ed',)


2

In [190]:
from sqlalchemy import func

In [191]:
session.query(func.count(User.name), User.name).group_by(User.name).all()

2018-03-18 07:46:50,858 INFO sqlalchemy.engine.base.Engine SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name


INFO:sqlalchemy.engine.base.Engine:SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name


2018-03-18 07:46:50,862 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[(1, 'ed'), (1, 'fred'), (1, 'mary'), (1, 'wendy')]

In [193]:
#To achieve our simple SELECT count(*) FROM table, we can apply it as:

In [194]:
session.query(func.count('*')).select_from(User).scalar()

2018-03-18 07:47:20,611 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT count(?) AS count_1 
FROM users


2018-03-18 07:47:20,616 INFO sqlalchemy.engine.base.Engine ('*',)


INFO:sqlalchemy.engine.base.Engine:('*',)


4

In [195]:
session.query(func.count(User.id)).scalar()

2018-03-18 07:47:31,757 INFO sqlalchemy.engine.base.Engine SELECT count(users.id) AS count_1 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT count(users.id) AS count_1 
FROM users


2018-03-18 07:47:31,761 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


4